# Function calling Using OpenAPI Using

1. Adding Function

2. Multiplication Function

3. Subraction Function

4. Division Function

5. Modulus Function
   
6. odd and Even Function

In [1]:
import os
import json
from openai import AzureOpenAI
from datetime import datetime
from zoneinfo import ZoneInfo
import pytz
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
endpoint = os.environ.get('ENDPOINT_URL')
deployment = os.environ.get('DEPLOYMENT_NAME')
api_key = os.environ.get('AZURE_OPENAI_KEY')

In [4]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-07-01-preview"
)

In [5]:
tools = [
    {
         "type": "function",
        "function": {
            "name": "add",
            "description": "Add two numbers together , if not any number given ask for number.",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number"},
                    "b": {"type": "number"}
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
        "name": "subtract",
        "description": "Subtract the second number from the first.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["a", "b"]
            }
        },

    },
    {
        "type": "function",
        "function": {
        "name": "multiply",
        "description": "Multiply two numbers together.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["a", "b"]
          },

       },
    },
    {
        "type": "function",
        "function": {
        "name": "divide",
        "description": "Divide the first number by the second.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["a", "b"]
           },

       },
    },
    {
        "type": "function",
        "function": {
        "name": "Odd",
        "description": "Provide wheather given number is odd  ",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number"},
                
            },
            "required": ["a"]
           },

       },
    },
    {
        "type": "function",
        "function": {
        "name": "Even",
        "description": "Provide Wheather given number is even ",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number"},
            },
            "required": ["a"]
           },

       },
    }
  
    
]


In [6]:
def add(a, b):
    return a+b;

def subtract(a,b):
    return a-b;

def multiply(a,b):
    return a*b;

def divide(a,b):
    return a / b;


In [7]:
def execute_function(function_name, function_args):
    if function_name == "add":
        return add(**function_args)
    elif function_name == "subtract":
        return subtract(**function_args)
    elif function_name == "multiply":
        return multiply(**function_args)
    elif function_name == "divide":
        return divide(**function_args)
    else:
        return "Function not recognized."

In [8]:
def call_openai_function_calling(messages, tools):
    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools = tools,
        tool_choice="auto",
        temperature=0
    )
    return response

In [21]:
# Function to get a chat response from OpenAI
def get_chat_response(prompt, messages):
    # Initialize messages with system role
    messages.append({"role": "user", "content": prompt})

    # Get the response from OpenAI
    response = call_openai_function_calling(messages, tools)
    response_message = response.choices[0].message
    messages.append(response_message)

    # Check if there is a function call
    function_call = response_message.tool_calls

    if function_call:
        # Get the function name and arguments
        function_name = function_call[0].function.name
        function_args = json.loads(function_call[0].function.arguments)

        # Execute the function and store the result
        function_result = execute_function(function_name, function_args)

        # Append the result of the function execution to the messages
        messages.append({
            "tool_call_id": function_call[0].id,
            "role": "tool",
            "name": function_name,
            "content": str(function_result),
        })
    else:
        # No function call, use the response content directly
        print("No tool calls were made by the model.")
        return response.choices[0].message.content

    # Second API call: Get the final response from the model
    final_response = call_openai_function_calling(messages, tools)

    return final_response.choices[0].message.content


In [26]:
# Main loop for interaction
def chat_main():
 messages = [
        {"role": "system", "content": "You are a helpful assistant."}
     ]
 while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chat session ended.")
        break
    response = get_chat_response(user_input, messages)
    print(f"Assistant: {response}")

chat_main()

You:  10+5


Assistant: The result of 10 + 5 is 15.


You:  quit


Chat session ended.
